# 01 - Data Preparation

This notebook will prepare data (synthetic fallback if we don't have Alpaca keys).

## Overview

- Set up seed and imports
- Load raw CSVs via `src.utils`
- Validate required columns and check missingness
- Rule-based sentiment on headlines; aggregate by date+symbol
- Merge prices, returns, sentiment into clean dataset


## Seed, Imports, and Paths


In [35]:
from pathlib import Path

import pandas as pd

from src.utils import set_seed, load_prices, load_returns, load_headlines, validate_columns
from src.features import add_sentiment_scores

# Reproducibility
set_seed(42)

# Resolve project root so relative paths work from notebooks/
from pathlib import Path as _P

_DEF_CANDIDATES = [_P.cwd(), _P.cwd().parent, _P.cwd().parent.parent]
for _base in _DEF_CANDIDATES:
    if (_base / "pyproject.toml").exists() and (_base / "data").exists():
        PROJECT_ROOT = _base
        break
else:
    PROJECT_ROOT = _P.cwd()

DATA_DIR = PROJECT_ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"


## Data Dictionary (Schemas)

- prices.csv: `date`, `ticker`, `sector`, `close`, `volume`, `volatility`
- returns.csv: `date`, `ticker`, `return`
- headlines.csv: `date`, `symbol`, `headline`, `source?`, `created_at?`


## Load and Validate Raw Data


In [36]:
prices = load_prices(RAW_DIR / "prices.csv")
returns = load_returns(RAW_DIR / "returns.csv")
headlines = load_headlines(RAW_DIR / "headlines.csv")

# Validate columns explicitly (defensive)
validate_columns(prices, ["date", "ticker", "sector", "close", "volume", "volatility"]) if not prices.empty else None
validate_columns(returns, ["date", "ticker", "return"]) if not returns.empty else None
validate_columns(headlines, ["date", "symbol", "headline"]) if not headlines.empty else None


## Missing Data Checks


In [37]:
def summarize_missing(df, name: str):
    if df.empty:
        print(f"{name}: EMPTY")
        return
    print(name)
    display(df.isna().sum())

summarize_missing(prices, "prices")
summarize_missing(returns, "returns")
summarize_missing(headlines, "headlines")


prices


date          0
ticker        0
sector        0
close         0
volume        0
volatility    0
dtype: int64

returns


date      0
ticker    0
return    0
dtype: int64

headlines


date          0
symbol        0
headline      0
source        0
created_at    0
dtype: int64

## Rule-based Sentiment Scoring and Aggregation


In [38]:
# Score sentiment per headline
headlines_scored = add_sentiment_scores(headlines) if not headlines.empty else headlines

# Aggregate by date + symbol (ticker)
if not headlines_scored.empty:
    agg = (
        headlines_scored
        .groupby(["date", "symbol"], as_index=False)
        .agg(sentiment_score=("sentiment_score", "mean"),
             n_headlines=("headline", "count"))
    )
else:
    import pandas as pd
    agg = pd.DataFrame(columns=["date", "symbol", "sentiment_score", "n_headlines"])


## Merge Dataset and Save


In [39]:
# Align symbol->ticker for join
if not agg.empty:
    agg = agg.rename(columns={"symbol": "ticker"})

# Left-join sentiment onto returns then prices for a tidy panel
merged = None
if not prices.empty and not returns.empty:
    pr = prices.merge(returns, on=["date", "ticker"], how="inner")
    merged = pr.merge(agg, on=["date", "ticker"], how="left") if not agg.empty else pr.copy()
elif not prices.empty:
    merged = prices.copy()
elif not returns.empty:
    merged = returns.copy()

# Write processed output (header-only if empty)
from src.utils import write_csv_safe
import pandas as pd

if merged is None:
    merged = pd.DataFrame(columns=["date","ticker","sector","close","volume","volatility","return","sentiment_score","n_headlines"]) 

write_csv_safe(merged, PROCESSED_DIR / "merged.csv", index=False)
